In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
import keras
import glob
import cv2
import os

#TRAINGING SET:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def getImages(no_of_classes=10):
    pest_images = []
    labels = []
    i = 0    
    for pest_image_path in glob.glob("/content/drive/MyDrive/ColabNotebooks/PestImages/pestRGB_256_256/*"):
        insect_label = pest_image_path.split("/")[-1]
        if no_of_classes == i:
            break
        for image_path in glob.glob(os.path.join(pest_image_path, "*.jpg")):
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            image = cv2.imread(image_path)            
            image = cv2.resize(image, (224, 224),interpolation = cv2.INTER_CUBIC) 
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            pest_images.append(image)
            labels.append(insect_label)
        i = i + 1

    pest_images = np.array(pest_images)
    labels = np.array(labels)
    label_to_id_dict = {v: i for i, v in enumerate(np.unique(labels))}
    id_to_label_dict = {v: k for k, v in label_to_id_dict.items()} 

    label_ids = np.array([label_to_id_dict[x] for x in labels])
    pest_images.shape, label_ids.shape, labels.shape
    
    print ("Total Images : " +  str(pest_images.shape[0]))
    # print ("Total Labels : " +  str(label_ids))    
    return pest_images, label_ids

In [ ]:
pests, labels = getImages(no_of_classes=10)

Total Images : 500


In [ ]:
#data augmentation
def Augmentation(X_train,X_valid,X_test,Y_train,Y_valid,Y_test,batch_size,test_batchSize):
       
        #Augmentation ,we will use for training and not in validation and testing
        train_datagen= ImageDataGenerator(rescale=1./255,
                                          rotation_range=30,
                                          width_shift_range=0.2,
                                          height_shift_range=0.2,
                                          shear_range=0.2,
                                          zoom_range=0.2,
                                          horizontal_flip=True)
        
        valid_datagen = ImageDataGenerator(rescale=1. / 255)
        test_datagen = ImageDataGenerator(rescale=1. / 255)

        train_batch = train_datagen.flow(X_train,
                                         Y_train,                                         
                                         batch_size=batch_size,shuffle=True)              
                                        
        valid_batch = valid_datagen.flow(X_valid,
                                         Y_valid,
                                         batch_size=batch_size,shuffle=True)            

        test_batch = test_datagen.flow(X_test,
                                       Y_test,                                       
                                       batch_size=test_batchSize,shuffle=False)              
                                       
        return (train_batch,valid_batch,test_batch)

In [ ]:
#SETTING UP InsceptionV3
def model(X_train, X_valid,X_test, Y_train, Y_valid, Y_test,Y,numClasses):
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    from keras.layers import Dense, Dropout, Flatten    
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    from keras.models import Sequential    
    from keras.models import Model
    
    inception=InceptionV3(input_shape=[224,224,3],weights='imagenet',include_top=False)

    #don't train the existing weights
    # for layer in inception.layers:
    #   layer.trainable=False
    # I chose to train the top 2 inception blocks, i.e. I will freeze the first 249 layers and unfreeze the rest:
    for layer in inception.layers[:263]:
      layer.trainable = False
    for layer in inception.layers[263:]:
      layer.trainable = True

    x=Flatten()(inception.output)    
    # x = Dense(1024, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    # x=Dense(64,activation='relu')(x)
    x=Dropout(rate=0.3)(x)
    prediction=Dense(numClasses,activation='softmax')(x)
    model=Model(inputs=inception.input,outputs=prediction)

    from tensorflow.keras.optimizers import SGD
    model.compile(optimizer=SGD(lr=0.0001, momentum=0.90), loss='categorical_crossentropy',metrics=['accuracy'])
  
    # opt = keras.optimizers.Adam(lr=0.0001)
    # model.compile(loss=keras.losses.categorical_crossentropy,
    #               optimizer=opt,
    #               metrics=['accuracy'])    

    batch_size = 25
    test_batchSize=1
    epochs = 50
    train_batch, valid_batch,test_batch = Augmentation(X_train,X_valid, X_test, Y_train, Y_valid, Y_test,batch_size,test_batchSize)
      
    model.fit(train_batch, 
              steps_per_epoch= X_train.shape[0]//batch_size,
              validation_data=valid_batch, 
              epochs=epochs,
              validation_steps=X_valid.shape[0]//batch_size, 
              verbose=2)
   
    test_accuracy=model.evaluate(test_batch,steps=len(test_batch))
    print('Testing Accuracy Score :',test_accuracy)
   
    return test_accuracy

In [ ]:
from sklearn.model_selection  import train_test_split,KFold
numClasses=10
train,test,trLabels,teLabels=train_test_split(pests,labels,test_size=0.2, random_state=42)
X_test = test
Y_test = teLabels    
Y=Y_test
Y_test = keras.utils.to_categorical(Y_test,numClasses)
def KFoldCV(Images, Labels, numClasses):
    # KFold Cross Validation approach
    kf = KFold(n_splits=8,shuffle=True,random_state=42)
    kf.split(Images)

    # Initialize the test accuracy array. The testing accuracy of each model will be added to this array
    test_accuracy_model = []
    Fold=1
    # Iterate over each train-test split
    for train_index, valid_index in kf.split(Images):          
          print('Fold No:', Fold)
        # Split training:validation        
          X_train, X_valid = Images[train_index], Images[valid_index]
          Y_train, Y_valid = Labels[train_index], Labels[valid_index]         

        #One Hot Encode the Output
          Y_train = keras.utils.to_categorical(Y_train,numClasses)
          Y_valid = keras.utils.to_categorical(Y_valid,numClasses)         

        # Train the model
          print('Original Sizes:', X_train.shape,X_valid.shape, X_test.shape,Y_train.shape,Y_valid.shape,Y_test.shape)
          score=model(X_train, X_valid,X_test, Y_train, Y_valid, Y_test,Y,numClasses)
          test_accuracy_model.append(score[1]*100)
    
          Fold=Fold+1
    return (test_accuracy_model)

#  8-Fold 10 Classes 

In [ ]:
import time
start_time= time.time()
accurary = KFoldCV(train,trLabels, 10)
end_time=time.time()
print(accurary)
print("%s %.2f%%" % ("Average Accuracy: ",np.mean(accurary)))
print("%s %.2f%%" % ("Standard Deviation: +/-", np.std(accurary)))
print("Total time taken {}".format(end_time-start_time))  


Fold No: 1
Original Sizes: (350, 224, 224, 3) (50, 224, 224, 3) (100, 224, 224, 3) (350, 10) (50, 10) (100, 10)
Epoch 1/50
14/14 - 13s - loss: 2.4559 - accuracy: 0.1000 - val_loss: 2.0945 - val_accuracy: 0.1800
Epoch 2/50
14/14 - 4s - loss: 2.0753 - accuracy: 0.2543 - val_loss: 1.5039 - val_accuracy: 0.6400
Epoch 3/50
14/14 - 4s - loss: 1.7232 - accuracy: 0.4943 - val_loss: 1.0401 - val_accuracy: 0.8400
Epoch 4/50
14/14 - 4s - loss: 1.3408 - accuracy: 0.6457 - val_loss: 0.7045 - val_accuracy: 0.8400
Epoch 5/50
14/14 - 4s - loss: 1.1264 - accuracy: 0.7000 - val_loss: 0.4690 - val_accuracy: 0.9000
Epoch 6/50
14/14 - 4s - loss: 0.8314 - accuracy: 0.8029 - val_loss: 0.3432 - val_accuracy: 0.9400
Epoch 7/50
14/14 - 4s - loss: 0.6834 - accuracy: 0.8400 - val_loss: 0.2904 - val_accuracy: 0.9400
Epoch 8/50
14/14 - 4s - loss: 0.5652 - accuracy: 0.8629 - val_loss: 0.2288 - val_accuracy: 0.9600
Epoch 9/50
14/14 - 4s - loss: 0.5041 - accuracy: 0.8943 - val_loss: 0.2079 - val_accuracy: 0.9600
Epoch